# CarList Listings

This project aims to analyse car listings data scraped from the Carlist.my website. Each group member was assigned a different range of pages to scrape, resulting in multiple datasets that will be merged into one unified dataset for further analysis. At this stage, the exact number of rows, columns, and memory usage is unknown, as the datasets have yet to be combined.

The project will involve essential data preprocessing steps including merging, cleaning, and transformation to ensure data consistency and usability. Following that, we will apply performance optimization techniques using various library. Two optimization methods will be implemented and compared against the baseline (non-optimized) version to observe their effect on speed and efficiency.

**Group Members:**
---

| Name          | Matrix Number | Library Used |
|---------------|---------------|---------------|
| Marcus Joey Sayner         | A22EC0193     | PySpark        |
| Muhammad Luqman Hakim bin Mohd Rizaudin           | A22EC0086     | Modin          |
| Goh Jing Yang       | A22EC0052     | Dask         |
| Camily Tang Jia Lei          | A22EC0039     | Pandas        |


## 1. Combine Dataset

To prepare the dataset for analysis, we first combined the individual files collected by each group member: Camily, Luqman, Jing Yang, and Marcus. Each member scraped a different range of pages from the Carlist.my website and saved the data as a CSV file. These files were then loaded using the pandas library and concatenated into one comprehensive dataset. This combined dataset is essential for performing consistent data cleaning, transformation, and performance optimization across all entries.

In [17]:
pip install pandas pymongo

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import pandas as pd
from pymongo import MongoClient

# List of group member files
files = [
    "camily_carlist_listings.csv",
    "luqman_carlist_listings.csv",
    "jingyang_carlist_listings.csv",
    "marcus_carlist_listings.csv"
]

# Read and combine datasets
dfs = [pd.read_csv(file) for file in files]
combined_df = pd.concat(dfs, ignore_index=True)

# Save combined dataset
combined_df.to_csv("carlist_combined.csv", index=False)

# Output shape and preview
print("Combined dataset shape:", combined_df.shape)
print(combined_df.head(5))

# Connect to MongoDB 
client = MongoClient("mongodb://localhost:27017")
db = client["carlist_db"]
collection = db["listings"]

# Insert combined data into MongoDB
collection.insert_many(combined_df.to_dict(orient="records"))
print("✅ Data inserted into MongoDB successfully.")


Combined dataset shape: (174150, 17)
                                            Car Name Car Brand      Car Model  \
0                   2023 Lexus RX350 2.4 F Sport SUV     Lexus          RX350   
1  2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...    Toyota         Estima   
2  2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...   Porsche        Cayenne   
3             2021 Honda City 1.5 V i-VTEC Hatchback     Honda           City   
4  2022 Toyota Corolla Cross 1.8 V SUV Full Servi...    Toyota  Corolla Cross   

   Manufacture Year  Body Type                Fuel Type            Mileage  \
0              2023        SUV  Petrol - Unleaded (ULP)     5 - 10K KM       
1              2010        MPV  Petrol - Unleaded (ULP)  115 - 120K KM       
2              2020      Coupe  Petrol - Unleaded (ULP)    20 - 25K KM       
3              2021  Hatchback  Petrol - Unleaded (ULP)    90 - 95K KM       
4              2022        SUV  Petrol - Unleaded (ULP)    80 - 85K KM       

  Trans

## 2. Data Preparation and Cleaning

In this phase of the project, we focus on preparing the combined dataset for analysis by cleaning and transforming the data. This step involves addressing common issues such as missing values, standardizing column names, removing irrelevant columns, and correcting any inconsistencies in the data. We will also handle data types to ensure that all columns are in the appropriate format for analysis. Proper data cleaning is essential to ensure the accuracy and consistency of the dataset before applying any optimization techniques or further analysis.

### Install and Import Libraries

**1. Pandas**

In [19]:
import pandas as pd
import numpy as np
import time
import psutil
from pymongo import MongoClient

**2. PySpark**

**3. Modin**

In [20]:
%pip install modin[dask] dask distributed

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.1 MB 1.4 MB/s eta 0:00:01
   - -------------------------------------- 0.0/1.1 MB 495.5 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.1 MB 939.4 kB/s eta 0:00:02
   -------- ------------------------------- 0.2/1.1 MB 1.4 MB/s eta 0:00:01
   ------------ --------------------------- 0.3/1.1 MB 1.6 MB/s eta 0:00:01
   ------------------------- -------------- 0.7/1.1 MB 2.7 MB/s eta 0:00:01
   ---------------------------- ----------- 0.8/1.1 MB 2.7 MB/s eta 0:00:01
   -------------------------------- ------- 0.9/1.1 MB 2.6 MB/s eta 0:00:01
   ---------------------------------------  1.1/1.1 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 2.6 MB/s eta 0:00:00
Note: you may need t

In [21]:
import os
os.environ["MODIN_ENGINE"] = "dask"

import modin.pandas as md
import psutil
import time

**4. Dask**

In [ ]:
%pip install dask
%pip install dask pymongo



In [15]:
import dask.dataframe as dd  # Dask for handling large dataframes
import pandas as pd         # Pandas for smaller, in-memory operations (optional)
import numpy as np 

### Import Data

**1. Pandas**

In [70]:
%%time

# Read from MongoDB into pandas DataFrame
data = list(collection.find())  # Fetch all documents as a list of dicts
df_pandas = pd.DataFrame(data)

# Drop the MongoDB-generated '_id' column
df_pandas.drop(columns=['_id'], inplace=True)

# Output preview
print("✅ Data loaded from MongoDB")
df_pandas.head()

✅ Data loaded from MongoDB
CPU times: total: 2.31 s
Wall time: 2.43 s


,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Seating Capacity,Location,Sales Channel,Seat Capacity,URL
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,5,"Selangor, Klang",Sales Agent ...,NaN,NaN
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,7,"Selangor, Kajang",Sales Agent ...,NaN,NaN
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,4,"Selangor, Klang",Sales Agent ...,NaN,NaN
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,5,"Johor, Ulu Tiram",Sales Agent ...,NaN,NaN
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,5,"Selangor, Kajang",Sales Agent ...,NaN,NaN


**2. PySpark**

**3. Modin**

In [6]:
%%time

# Read from MongoDB into pandas DataFrame
data = list(collection.find())  # Fetch all documents as a list of dicts
df_modin = md.DataFrame(data)

# Drop the MongoDB-generated '_id' column
df_modin.drop(columns=['_id'], inplace=True)

# Output preview
print("Data loaded from MongoDB")
df_modin.head()

Perhaps you already have a cluster running?
Hosting the HTTP server on port 49997 instead


Data loaded from MongoDB
CPU times: total: 6.28 s
Wall time: 22.2 s


,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Seating Capacity,Location,Sales Channel,Seat Capacity,URL
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,5,"Selangor, Klang",Sales Agent ...,NaN,NaN
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,7,"Selangor, Kajang",Sales Agent ...,NaN,NaN
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,4,"Selangor, Klang",Sales Agent ...,NaN,NaN
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,5,"Johor, Ulu Tiram",Sales Agent ...,NaN,NaN
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,5,"Selangor, Kajang",Sales Agent ...,NaN,NaN


**4. Dask**

In [13]:
# %%time (if you're using Jupyter notebook to check execution time)

# Read data from CSV file into Dask DataFrame (similar to Pandas)
df_dask = dd.read_csv('path_to_your_data.csv')

# Drop a column (just like in the MongoDB example)
# Replace 'column_name' with the actual column you want to drop (e.g., '_id')
df_dask = df_dask.drop(columns=['column_name'])

# Output preview (since Dask is lazy, we need to compute or take a sample)
print("✅ Data loaded into Dask DataFrame")
print(df_dask.head())  # Show the first few rows

FileNotFoundError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: [WinError 2] The system cannot find the file specified: 'c:/Users/Honor/OneDrive - Universiti Teknologi Malaysia (UTM)/Desktop/final/final/path_to_your_data.csv'

### Combine Columns

To standardize the data, we combined the 'Seat Capacity' and 'Seating Capacity' columns into one, prioritizing the non-null values from each.

**1. Pandas**

In [71]:
%%time

# Combine 'Seat Capacity' and 'Seating Capacity' into one column
df_pandas['Seat Capacity'] = df_pandas['Seat Capacity'].combine_first(df_pandas['Seating Capacity'])

# Drop 'Seating Capacity' column
df_pandas.drop(columns=['Seating Capacity'], inplace=True)


CPU times: total: 62.5 ms
Wall time: 51.7 ms


In [72]:
# Display dataframe
df_pandas.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity,URL
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,5,NaN
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,7,NaN
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,4,NaN
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,"Johor, Ulu Tiram",Sales Agent ...,5,NaN
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,5,NaN


**3. Modin**

In [8]:
%%time

df_modin['Seat Capacity'] = df_modin['Seat Capacity'].fillna(df_modin['Seating Capacity'])
df_modin.drop(columns=['Seating Capacity'], inplace=True)

CPU times: total: 125 ms
Wall time: 170 ms


In [9]:
df_modin.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity,URL
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,5,NaN
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,7,NaN
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,4,NaN
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,"Johor, Ulu Tiram",Sales Agent ...,5,NaN
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,5,NaN


### Remove Unwanted Column(s)

To clean the dataset, we removed the unwanted 'URL' column, which was irrelevant to the analysis.

**1. Pandas**

In [73]:
%%time

# Remove the 'URL' column
df_pandas.drop(columns=['URL'], inplace=True)

CPU times: total: 46.9 ms
Wall time: 41.7 ms


In [74]:
# Display dataframe
df_pandas.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,"Johor, Ulu Tiram",Sales Agent ...,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,5


**3. Modin**

In [11]:
%%time

# Remove the 'URL' column
df_modin.drop(columns=['URL'], inplace=True)

CPU times: total: 0 ns
Wall time: 2.99 ms


In [12]:
df_modin.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10K KM,Automatic,Black,375000,"RM 4,862/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120K KM,Automatic,White,55999,RM 726/month,http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25K KM,Automatic,Grey,662222,"RM 8,585/month",http://schema.org/RefurbishedCondition,"Selangor, Klang",Sales Agent ...,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95K KM,Automatic,Silver,67000,RM 869/month,http://schema.org/UsedCondition,"Johor, Ulu Tiram",Sales Agent ...,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85K KM,Automatic,White,98999,"RM 1,283/month",http://schema.org/UsedCondition,"Selangor, Kajang",Sales Agent ...,5


### Replace Null Values

To handle missing values in the dataset, we replaced the null values in the categorical columns with the most frequent value (mode) of each respective column.

**1. Pandas**

In [75]:
df_pandas.isnull().sum()


Car Name             0
Car Brand            0
Car Model            0
Manufacture Year     0
Body Type           32
Fuel Type           42
Mileage              0
Transmission         0
Color                0
Price                0
Installment          0
Condition            0
Location             0
Sales Channel       53
Seat Capacity        0
dtype: int64

In [76]:
%%time

# Replace missing values in categorical columns with the mode (most frequent value)
df_pandas['Body Type'] = df_pandas['Body Type'].fillna(df_pandas['Body Type'].mode()[0])
df_pandas['Fuel Type'] = df_pandas['Fuel Type'].fillna(df_pandas['Fuel Type'].mode()[0])
df_pandas['Sales Channel'] = df_pandas['Sales Channel'].fillna(df_pandas['Sales Channel'].mode()[0])

# Replace '-' with the mode for 'Seat Capacity'
df_pandas['Seat Capacity'] = df_pandas['Seat Capacity'].replace('-', df_pandas['Seat Capacity'].mode()[0])

CPU times: total: 172 ms
Wall time: 172 ms


In [77]:
df_pandas.isnull().sum()

Car Name            0
Car Brand           0
Car Model           0
Manufacture Year    0
Body Type           0
Fuel Type           0
Mileage             0
Transmission        0
Color               0
Price               0
Installment         0
Condition           0
Location            0
Sales Channel       0
Seat Capacity       0
dtype: int64

### String Cleaning and Formatting

**3. Modin**

In [13]:
df_modin.isnull().sum()

Car Name             0
Car Brand            0
Car Model            0
Manufacture Year     0
Body Type           32
Fuel Type           42
Mileage              0
Transmission         0
Color                0
Price                0
Installment          0
Condition            0
Location             0
Sales Channel       53
Seat Capacity        0
dtype: int64

In [14]:
%%time

# Replace missing values in categorical columns with the mode (most frequent value)
df_modin['Body Type'] = df_modin['Body Type'].fillna(df_modin['Body Type'].mode()[0])
df_modin['Fuel Type'] = df_modin['Fuel Type'].fillna(df_modin['Fuel Type'].mode()[0])
df_modin['Sales Channel'] = df_modin['Sales Channel'].fillna(df_modin['Sales Channel'].mode()[0])

# Replace '-' with the mode for 'Seat Capacity'
df_modin['Seat Capacity'] = df_modin['Seat Capacity'].replace('-', df_modin['Seat Capacity'].mode()[0])

CPU times: total: 4.25 s
Wall time: 6.84 s


In [15]:
df_modin.isnull().sum()

Car Name            0
Car Brand           0
Car Model           0
Manufacture Year    0
Body Type           0
Fuel Type           0
Mileage             0
Transmission        0
Color               0
Price               0
Installment         0
Condition           0
Location            0
Sales Channel       0
Seat Capacity       0
dtype: int64

In this part, we clean string-based columns to ensure consistent formatting and remove unnecessary characters or metadata.

**1. Pandas**

In [78]:
## Part 1: Clean Installment column
start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

df_pandas['Installment'] = df_pandas['Installment'].str.replace('RM', '', regex=False)
df_pandas['Installment'] = df_pandas['Installment'].str.replace(',', '', regex=False)
df_pandas['Installment'] = df_pandas['Installment'].str.replace('/month', '', regex=False)
df_pandas['Installment'] = pd.to_numeric(df_pandas['Installment'], errors='coerce')

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time: 238 ms, CPU time: 375 ms


In [79]:
## Part 2: Extract Clean Values from Condition column

start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

# Extract the condition from the URL if it exists, otherwise leave as is
df_pandas['Condition'] = df_pandas['Condition'].apply(lambda x: x.split('/org/')[-1].split('Condition')[0] if 'http' in str(x) else x)

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time: 51 ms, CPU time: 78 ms


In [ ]:
## Part 3: Clean Sales Channel

start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

df_pandas['Sales Channel'] = df_pandas['Sales Channel'].str.extract(r'^(Sales Agent|Dealer)')

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time:197 ms, CPU time: 359 ms


In [81]:
## Part 4: Clean Mileage

start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

# Remove "K KM"
df_pandas['Mileage'] = df_pandas['Mileage'].str.replace('K KM', '', regex=False)

# Clean and standardize mileage into 5K step ranges
def process_mileage(mileage):
    if not isinstance(mileage, str) or mileage.strip() == '':
        return None
    try:
        if '-' in mileage:
            start, end = mileage.split('-')
            return f"{int(start.strip())} - {int(end.strip())}"
        else:
            val = int(mileage.strip())
            lower = (val // 5) * 5
            upper = lower + 5
            return f"{lower} - {upper}"
    except:
        return None

df_pandas['Mileage'] = df_pandas['Mileage'].apply(process_mileage)

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time: 215 ms, CPU time: 406 ms


In [82]:
# Display dataframe
df_pandas.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10,Automatic,Black,375000,4862,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120,Automatic,White,55999,726,http://schema.org/Used,"Selangor, Kajang",Sales Agent,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25,Automatic,Grey,662222,8585,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95,Automatic,Silver,67000,869,http://schema.org/Used,"Johor, Ulu Tiram",Sales Agent,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85,Automatic,White,98999,1283,http://schema.org/Used,"Selangor, Kajang",Sales Agent,5


**3. Modin**

In [20]:
## Part 1: Clean Installment column

start_time = time.time()
start_cpu_time = psutil.cpu_times().user

df_modin['Installment'] = df_modin['Installment'].str.replace('RM', '', regex=False)
df_modin['Installment'] = df_modin['Installment'].str.replace(',', '', regex=False)
df_modin['Installment'] = df_modin['Installment'].str.replace('/month', '', regex=False)
df_modin['Installment'] = pd.to_numeric(df_modin['Installment'], errors='coerce')

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")

Wall time: 1643 ms, CPU time: 5172 ms


In [21]:
## Part 2: Extract Clean Values from Condition column
start_time = time.time()
start_cpu_time = psutil.cpu_times().user

# Extract the condition from the URL if it exists, otherwise leave as is
df_modin['Condition'] = df_modin['Condition'].apply(lambda x: x.split('/org/')[-1].split('Condition')[0] if 'http' in str(x) else x)

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")

Wall time: 131 ms, CPU time: 469 ms


In [22]:
## Part 3: Clean Sales Channel

start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

df_modin['Sales Channel'] = df_modin['Sales Channel'].str.extract(r'^(Sales Agent|Dealer)')

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time: 907 ms, CPU time: 2203 ms


In [23]:
## Part 4: Clean Mileage
start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

# Remove "K KM"
df_modin['Mileage'] = df_modin['Mileage'].str.replace('K KM', '', regex=False)

# Clean and standardize mileage into 5K step ranges
def process_mileage(mileage):
    if not isinstance(mileage, str) or mileage.strip() == '':
        return None
    try:
        if '-' in mileage:
            start, end = mileage.split('-')
            return f"{int(start.strip())} - {int(end.strip())}"
        else:
            val = int(mileage.strip())
            lower = (val // 5) * 5
            upper = lower + 5
            return f"{lower} - {upper}"
    except:
        return None

df_modin['Mileage'] = df_modin['Mileage'].apply(process_mileage)

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time: 677 ms, CPU time: 3125 ms


In [24]:
df_modin.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage,Transmission,Color,Price,Installment,Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10,Automatic,Black,375000,4862,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120,Automatic,White,55999,726,http://schema.org/Used,"Selangor, Kajang",Sales Agent,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25,Automatic,Grey,662222,8585,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95,Automatic,Silver,67000,869,http://schema.org/Used,"Johor, Ulu Tiram",Sales Agent,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85,Automatic,White,98999,1283,http://schema.org/Used,"Selangor, Kajang",Sales Agent,5


### Rename Columns

Renamed selected columns to include units for better understanding and presentation.

**1. Pandas**

In [83]:
%%time

df_pandas.rename(columns={
    'Mileage': 'Mileage (K KM)',
    'Price': 'Price (RM)',
    'Installment': 'Installment (RM)'
}, inplace=True)

CPU times: total: 0 ns
Wall time: 317 μs


In [84]:
# Display dataframe
df_pandas.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage (K KM),Transmission,Color,Price (RM),Installment (RM),Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10,Automatic,Black,375000,4862,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120,Automatic,White,55999,726,http://schema.org/Used,"Selangor, Kajang",Sales Agent,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25,Automatic,Grey,662222,8585,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95,Automatic,Silver,67000,869,http://schema.org/Used,"Johor, Ulu Tiram",Sales Agent,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85,Automatic,White,98999,1283,http://schema.org/Used,"Selangor, Kajang",Sales Agent,5


**3. Modin**

In [25]:
%%time

df_modin.rename(columns={
    'Mileage': 'Mileage (K KM)',
    'Price': 'Price (RM)',
    'Installment': 'Installment (RM)'
}, inplace=True)

CPU times: total: 0 ns
Wall time: 998 μs


In [26]:
df_modin.head()

,Car Name,Car Brand,Car Model,Manufacture Year,Body Type,Fuel Type,Mileage (K KM),Transmission,Color,Price (RM),Installment (RM),Condition,Location,Sales Channel,Seat Capacity
0,2023 Lexus RX350 2.4 F Sport SUV,Lexus,RX350,2023,SUV,Petrol - Unleaded (ULP),5 - 10,Automatic,Black,375000,4862,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,5
1,2010 Toyota Estima 2.4 Aeras MPV Hot Mpv Car I...,Toyota,Estima,2010,MPV,Petrol - Unleaded (ULP),115 - 120,Automatic,White,55999,726,http://schema.org/Used,"Selangor, Kajang",Sales Agent,7
2,2020 Porsche Cayenne Coupe 4.0 V8 Turbo AWD Un...,Porsche,Cayenne,2020,Coupe,Petrol - Unleaded (ULP),20 - 25,Automatic,Grey,662222,8585,http://schema.org/Refurbished,"Selangor, Klang",Sales Agent,4
3,2021 Honda City 1.5 V i-VTEC Hatchback,Honda,City,2021,Hatchback,Petrol - Unleaded (ULP),90 - 95,Automatic,Silver,67000,869,http://schema.org/Used,"Johor, Ulu Tiram",Sales Agent,5
4,2022 Toyota Corolla Cross 1.8 V SUV Full Servi...,Toyota,Corolla Cross,2022,SUV,Petrol - Unleaded (ULP),80 - 85,Automatic,White,98999,1283,http://schema.org/Used,"Selangor, Kajang",Sales Agent,5


### Optimize Memory Usage by Using Efficient Data Types

The dataset may contain columns with default data types that are not memory-efficient. To improve performance and reduce memory consumption, we will convert columns to more suitable data types, such as downcasting numerical columns and converting categorical data to the category type.

**1. Pandas**

In [85]:
# Check the initial info about the dataset
df_pandas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174150 entries, 0 to 174149
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Car Name          174150 non-null  object
 1   Car Brand         174150 non-null  object
 2   Car Model         174150 non-null  object
 3   Manufacture Year  174150 non-null  int64 
 4   Body Type         174150 non-null  object
 5   Fuel Type         174150 non-null  object
 6   Mileage (K KM)    168791 non-null  object
 7   Transmission      174150 non-null  object
 8   Color             174150 non-null  object
 9   Price (RM)        174150 non-null  int64 
 10  Installment (RM)  174150 non-null  int64 
 11  Condition         174150 non-null  object
 12  Location          174150 non-null  object
 13  Sales Channel     169939 non-null  object
 14  Seat Capacity     174150 non-null  object
dtypes: int64(3), object(12)
memory usage: 19.9+ MB


In [86]:
# Calculate initial memory usage in MB
start_mem = df_pandas.memory_usage().sum() / 1024**2  # Convert bytes to megabytes
print(f"Initial memory usage: {start_mem:.2f} MB")

Initial memory usage: 19.93 MB


In [87]:
%%time

# Numeric downcasting
df_pandas['Manufacture Year'] = df_pandas['Manufacture Year'].astype('int16')
df_pandas[['Price (RM)', 'Installment (RM)']] = df_pandas[['Price (RM)', 'Installment (RM)']].astype('int32')
df_pandas['Seat Capacity'] = pd.to_numeric(df_pandas['Seat Capacity'], errors='coerce').astype('int8')

# Categorical conversion
categorical_columns = ['Car Brand', 'Car Model', 'Body Type', 'Fuel Type',
                       'Transmission', 'Color', 'Condition', 'Sales Channel', 'Mileage (K KM)']
df_pandas[categorical_columns] = df_pandas[categorical_columns].apply(lambda x: x.astype('category'))

CPU times: total: 359 ms
Wall time: 356 ms


In [88]:
# Check the data types of all columns after the conversion
df_pandas.dtypes

Car Name              object
Car Brand           category
Car Model           category
Manufacture Year       int16
Body Type           category
Fuel Type           category
Mileage (K KM)      category
Transmission        category
Color               category
Price (RM)             int32
Installment (RM)       int32
Condition           category
Location              object
Sales Channel       category
Seat Capacity           int8
dtype: object

In [89]:
# Calculate memory usage after the conversion
end_mem = df_pandas.memory_usage().sum() / 1024**2  # Convert bytes to megabytes
print(f"Memory usage after optimization: {end_mem:.2f} MB")

# Print the memory saved
print(f"Memory saved: {start_mem - end_mem:.2f} MB")

Memory usage after optimization: 6.19 MB
Memory saved: 13.74 MB


**3. Modin**

In [27]:
# Check the initial info about the dataset
df_modin.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 174150 entries, 0 to 174149
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Car Name          174150 non-null  object
 1   Car Brand         174150 non-null  object
 2   Car Model         174150 non-null  object
 3   Manufacture Year  174150 non-null  int64 
 4   Body Type         174150 non-null  object
 5   Fuel Type         174150 non-null  object
 6   Mileage (K KM)    168791 non-null  object
 7   Transmission      174150 non-null  object
 8   Color             174150 non-null  object
 9   Price (RM)        174150 non-null  int64 
 10  Installment (RM)  174150 non-null  int64 
 11  Condition         174150 non-null  object
 12  Location          174150 non-null  object
 13  Sales Channel     169939 non-null  object
 14  Seat Capacity     174150 non-null  object
dtypes: int64(3), object(12)
memory usage: 19.9+ MB


In [28]:
# Calculate initial memory usage in MB
start_mem = df_modin.memory_usage().sum() / 1024**2  # Convert bytes to megabytes
print(f"Initial memory usage: {start_mem:.2f} MB")

Initial memory usage: 19.93 MB


In [29]:
%%time

# Numeric downcasting
df_modin['Manufacture Year'] = df_modin['Manufacture Year'].astype('int16')
df_modin[['Price (RM)', 'Installment (RM)']] = df_modin[['Price (RM)', 'Installment (RM)']].astype('int32')
df_modin['Seat Capacity'] = pd.to_numeric(df_modin['Seat Capacity'], errors='coerce').astype('int8')

# Categorical conversion
categorical_columns = ['Car Brand', 'Car Model', 'Body Type', 'Fuel Type',
                       'Transmission', 'Color', 'Condition', 'Sales Channel', 'Mileage (K KM)']
df_modin[categorical_columns] = df_modin[categorical_columns].apply(lambda x: x.astype('category'))

CPU times: total: 4.17 s
Wall time: 5.15 s


In [30]:
# Calculate memory usage after the conversion
end_mem = df_modin.memory_usage().sum() / 1024**2  # Convert bytes to megabytes
print(f"Memory usage after optimization: {end_mem:.2f} MB")

# Print the memory saved
print(f"Memory saved: {start_mem - end_mem:.2f} MB")

Memory usage after optimization: 6.27 MB
Memory saved: 13.66 MB


In [31]:
# Check the initial info about the dataset
df_modin.info()

<class 'modin.pandas.dataframe.DataFrame'>
RangeIndex: 174150 entries, 0 to 174149
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype   
---  ------            --------------   -----   
 0   Car Name          174150 non-null  object  
 1   Car Brand         174150 non-null  category
 2   Car Model         174150 non-null  category
 3   Manufacture Year  174150 non-null  int16   
 4   Body Type         174150 non-null  category
 5   Fuel Type         174150 non-null  category
 6   Mileage (K KM)    168791 non-null  category
 7   Transmission      174150 non-null  category
 8   Color             174150 non-null  category
 9   Price (RM)        174150 non-null  int32   
 10  Installment (RM)  174150 non-null  int32   
 11  Condition         174150 non-null  category
 12  Location          174150 non-null  object  
 13  Sales Channel     169939 non-null  category
 14  Seat Capacity     174150 non-null  int8    
dtypes: category(9), int16(1), int32(2), int8(1), o

### Drop Duplicated Rows

**1. Pandas**

In [90]:
# Check for duplicate rows (without %%time)
duplicates = df_pandas.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3940


In [ ]:
# Drop duplicate rows

start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

df_pandas = df_pandas.drop_duplicates()

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")


Wall time:138 ms, CPU time: 156 ms


In [92]:
# Check for duplicate rows after dropping
duplicates_after = df_pandas.duplicated().sum()
print(f"Number of duplicate rows after dropping: {duplicates_after}")

Number of duplicate rows after dropping: 0


**3. Modin**

In [32]:
# Check for duplicate rows (without %%time)
duplicates = df_modin.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 3940


In [36]:
# Drop duplicate rows
start_time = time.time()
start_cpu_time = psutil.cpu_times().user  # Start CPU time

df_modin = df_modin.drop_duplicates()

end_time = time.time()
end_cpu_time = psutil.cpu_times().user  # End CPU time

wall_time = (end_time - start_time) * 1000  # Wall time in ms
cpu_time = (end_cpu_time - start_cpu_time) * 1000  # CPU time in ms

print(f"Wall time: {wall_time:.0f} ms, CPU time: {cpu_time:.0f} ms")

Wall time: 3249 ms, CPU time: 10578 ms


In [37]:
# Check for duplicate rows (without %%time)
duplicates = df_modin.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0
